In [ ]:
def em(data, priors0, mu0, sigma0):
    loglik_threshold = 1e-10
    
    (nbVar,nbData) = data.shape
    nbStates = sigma0.shape[2]
    loglik_old=-1.797693134862316e308
    nbStep = 0;
    
    mu = mu0
    sigma = sigma0
    priors = priors0
    
    pxi = np.empty((nbData,nbStates))
    while True
    #E-step
        for i in range(0,nbStates): #Compute probability p(x|i)
            pxi[:,i] = gaussPDF(data, mu[:,i], sigma[:,:,i])
        #Compute posterior probability p(i|x)
        pix_tmp =  np.tile(priors,(nbData,1))*pxi
        pix = pix_tmp / np.tile(np.mat(np.sum(pix_tmp,axis=1)).T,(1,nbStates))
        #Compute cumulated posterior probability
        e = np.sum(pix,axis=0)
    #M-step
        for i in range(0,nbStates):
            #Update the priors
            priors[i] = e[i] / nbData;
            #Update the centers
            mu[:,i] = np.dot(data,pix[:,i]) / e[i]
            #Update the covariance matrices
            data_tmp1 = data - np.tile(mu[:,i],(1,nbData));
            sigma[:,:,i] = (np.tile(pix[:,i]',nbVar, 1) .* Data_tmp1*Data_tmp1') / e[i]
            #Add a tiny variance to avoid numerical instability
            sigma[:,:,i] = sigma[:,:,i] + 1e-5.*diag(ones(nbVar,1))

# 采用K-means初始化参数函数

In [2]:
from sklearn.cluster import KMeans 
import numpy as np

def em_init_kmeans(data,nbStates):
    (nbVar,nbData) = data.shape
    kmeans = KMeans(n_clusters=nbStates).fit(data.T)
    data_id = kmeans.labels_
    centers = kmeans.cluster_centers_
    mu = centers.T
    priors = np.empty((1,nbStates))
    sigma = np.empty((nbVar,nbVar,nbStates))
    priors = np.empty((1,nbStates))
    sigma = np.empty((nbVar,nbVar,nbStates))
    for i in range(0,nbStates):
        idtmp = np.where(data_id == i)
        priors[0,i] = len(idtmp)
        sigma[:,:,i] = np.cov(np.concatenate((data[:,idtmp[0]],data[:,idtmp[0]]),axis=1))
        sigma[:,:,i] = sigma[:,:,i] + (1e-5)*np.diag(np.ones((1,nbVar))[0])
    priors = priors / np.sum(priors)
    return priors,mu,sigma

# 多维高斯分布函数

In [3]:
import math
def gaussPDF(data, mu, sigma):
    data = np.array(np.mat(data))
    mu = np.array(np.mat(mu))
    sigma = np.array(np.mat(sigma))
    
    (nbVar,nbData) = data.shape
    data = data.T - np.tile(mu.T,(nbData,1));
    prob = np.sum(np.dot(data,np.linalg.inv(sigma))*data,axis=1)
    prob = np.exp(-0.5*prob) / np.sqrt((2*math.pi)**nbVar * (np.absolute(np.linalg.det(sigma))+2.2251e-308))
    return prob

# 读取数据并初始化参数

In [4]:
import scipy.io as scio
#可以使用scipy.io中的函数loadmat()读取mat文件
import numpy as np
from sklearn.cluster import KMeans

nbStates=4

dataFile = './data/data1.mat'
data = scio.loadmat(dataFile)['Data']

#priors,mu,sigma = em_init_kmeans(data,nbStates)


# 高斯混合模型

In [5]:
from sklearn.mixture import GaussianMixture
gmm = GaussianMixture(n_components = nbStates).fit(data.T)

# 高斯回归

In [9]:
def gmr(priors, mu, sigma, x, iN, out):
    nbData = x.size
    nbVar = mu.shape[1]
    nbStates = sigma.shape[0]

    pxi = np.empty((nbData, nbStates))
    for i in range(0, nbStates):
        pxi[:, i] = priors[i]*gaussPDF(x, mu[i, iN], sigma[i, iN, iN])
    beta = pxi/np.tile(np.sum(pxi, axis=1).reshape(-1, 1), (1, nbStates))
    y_tmp = np.empty((2, 100, 4))
    for j in range(0, nbStates):
        y_tmp[:, :, j] = np.tile(mu[j, out].reshape(-1, 1), (1, nbData)) + np.dot(np.dot(sigma[j, out, iN].reshape(-1, 1),
                                                                                         np.linalg.inv(np.array(np.mat(sigma[j, iN, iN])))), (x-np.tile(mu[j, iN], (1, nbData))))
    betaTmp=[]
    betaTmp.append(beta)
    betaTmp.append(beta)
    beta_tmp = np.array(betaTmp)
    y_tmp2 =beta_tmp * y_tmp;
    y = np.sum(y_tmp2,2)                                                                              
    
    return y, sigma_y

In [10]:
x = np.linspace(1,100,100)
mu = gmm.means_
sigma = gmm.covariances_
priors = gmm.weights_
iN = [0]
out = [1,2]

In [11]:
nbData = x.size
nbVar = mu.shape[1]
nbStates = sigma.shape[0]

pxi = np.empty((nbData, nbStates))
for i in range(0, nbStates):
    pxi[:, i] = priors[i]*gaussPDF(x, mu[i, iN], sigma[i, iN, iN])
beta = pxi/np.tile(np.sum(pxi, axis=1).reshape(-1, 1), (1, nbStates))
y_tmp = np.empty((2, 100, 4))
for j in range(0, nbStates):
    y_tmp[:, :, j] = np.tile(mu[j, out].reshape(-1, 1), (1, nbData)) + np.dot(np.dot(sigma[j, out, iN].reshape(-1, 1),
                                                                                     np.linalg.inv(np.array(np.mat(sigma[j, iN, iN])))), (x-np.tile(mu[j, iN], (1, nbData))))

In [12]:
betaTmp=[]
betaTmp.append(beta)
betaTmp.append(beta)
beta_tmp = np.array(betaTmp)
y_tmp2 =beta_tmp * y_tmp;
y = np.sum(y_tmp2,2)

In [14]:
import matplotlib.pyplot as plt
%matplotlib widget
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(data[0,0:100],y[0,:])
ax.scatter(data[0,0:100],data[1,0:100])
ax.scatter(data[0,0:100],y[1,:])
ax.scatter(data[0,0:100],data[2,0:100])
plt.legend(['预测1','实际1','预测2','实际2'])
plt.show

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<function matplotlib.pyplot.show(block=None)>

In [15]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(data[1,0:100],data[2,0:100])
ax.scatter(y[0,0:100],y[1,0:100])
plt.show

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<function matplotlib.pyplot.show(block=None)>